# Problem

There is an integer array `nums` sorted in ascending order (with **distinct** values).

Prior to being passed to your function, `nums` is **possibly rotated** at an unknown pivot index `k` (`1 <= k < nums.length`) such that the resulting array is `[nums[k], nums[k+1], ..., nums[n-1], nums[0], nums[1], ..., nums[k-1]]` (**0-indexed**). For example, `[0,1,2,4,5,6,7]` might be rotated at pivot index `3` and become `[4,5,6,7,0,1,2]`.

Given the array `nums` **after** the possible rotation and an integer `target`, return *the index of* `target` *if it is in* `nums`*, or* `-1` *if it is not in* `nums`.

You must write an algorithm with `O(log n)` runtime complexity.

 

**Example 1:**

```
Input: nums = [4,5,6,7,0,1,2], target = 0
Output: 4
```

**Example 2:**

```
Input: nums = [4,5,6,7,0,1,2], target = 3
Output: -1
```

**Example 3:**

```
Input: nums = [1], target = 0
Output: -1
```

 

**Constraints:**

- `1 <= nums.length <= 5000`
- `-10^4 <= nums[i] <= 10^4`
- All values of `nums` are **unique**.
- `nums` is an ascending array that is possibly rotated.
- `-10^4 <= target <= 10^4`

# Summary

This problem is a medium problem, and it implements a rule-based binary search instead of a naive binary search based on a sorted array. I should note the nature of the algorithm instead of the algorithm itself, such as the core of the binary search algorithm is to cut the undesired part and to search in the desired part in a sorted array.

Meanwhile, search element(s) in the sorted array can be generalized as searching elements in a consistent array which we can determine go which side.

# Problem Description 

Find the key in a rotated sorted array.

The key issue is how to sort the rotated array, then we can implement binary search to find the key.

And there is anther key issue that should we format a new array or using the raw array?

This problem is kinda interesting. It examines two points:

+ the generality of the binary search, which is drop one side by rules;
+ the consistency of the sequence, which promises a local sorted array.

# Methods

Binary search.

## Method 1 Linear Sort and Binary Search

+ **Time Complexity**: 
	+ Best case: $O(n)$
	+ Worst case: $O(n)$
+ **Space Complexity**: $O(n)$

Using linear sort to sort the array and using binary search to find the index of the target (key).

+ linear sort algorithm find the rotated element one by one, and it needs to avoid search length 1 array;
+ `list.index(key)` to return the index of the target at the `nums`.

Version 1 Naive

In [ ]:
class Solution:
    def search(self, nums: List[int], target: int) -> int:
        new_nums = []
        for i in range(len(nums) - 1):
            if nums[i] > nums[i + 1]:
                new_nums = nums[i + 1:]
                new_nums.extend(nums[:i + 1])
                break

        if not new_nums:  # avoid length 1 array
            new_nums = nums

        left = 0
        right = len(new_nums) - 1

        while left <= right:
            mid = (left + right) // 2
            if target == new_nums[mid]:
                return nums.index(target)
            elif target < new_nums[mid]:
                right = mid - 1
            else:
                left = mid + 1

        return -1

Version 2 Advanced

In [ ]:
class Solution:
    def search(self, nums: List[int], target: int) -> int:
        if target not in nums:
            return -1
        else:
            return nums.index(target)

## Method 2 Binary Sort and Binary Search [DEPRECATED]

Q: Can we just use binary search to sort and find the element?

A: Yes. We can implement the binary search three times to complete the problem. The first is to find the rotated boundary, and the last two is to find the target within the former and latter sub-array. Since this array is a possible rotated array, two times may be enough.

Q: How to use binary search to sort the array?

A: As the problem states the array is a possible rotated array, thus the search sort array may need to check each element instead of just half of the array.

Q: Is it necessary to sort the rotated array?

A: I think it is necessary, since binary search works in the sorted array. If I just use binary search to find the target and the length of the rotated part is less than the half of the length of the whole array, this part will not be checked and the algorithm will return the wrong answer, such as `[4, 1, 2, 3]`. **I don't know how to sort the array in $O(logn)$.**

In [ ]:
class Solution:
    def search(self, nums: List[int], target: int) -> int:
        def binary_search(nums: List[int], target: int) -> int:
            left = 0
            right = len(nums) - 1

            while left <= right:
                mid = (left + right) // 2
                if nums[mid] == target:
                    return mid
                elif nums[mid] < target:
                    left = mid + 1
                else:
                    right = mid - 1

            return -1

        left = 0
        right = len(nums) - 1
        boundary = None

        if right == left:  # length 1 array
            if nums[0] == target:
                return 0
            else:
                return -1

        while left <= right:  # length >2 array
            mid = (left + right) // 2
            if nums[mid - 1] > nums[mid]:
                boundary = mid
                break

        if boundary == None:
            return binary_search(nums, target)
        else:
            former = binary_search(nums[:boundary], target)
            latter = binary_search(nums[boundary:], target)
            if former != -1:
                return former
            else:
                return latter

## Method 3 Binary Search

+ **Time Complexity**: 
	+ Best case: $O(logn)$
	+ Worst case: $O(logn)$
+ **Space Complexity**: $O(1)$

Consider the scenarios:
+ Shorted array;
+ Rotated array:
    + rotated part exceeds the half, such as `[5, 1, 2, 3]` and `[4, 5, 6, 1]`.
    + rotated part equals the half, such as `[5, 1]`.

Consider the corresponding algorithms:
+ For the shorted array, naive binary search is enough;
+ For the rotated array, we need to check both sides and to determine which side is the target side.
    + If the former exceeds the half, the median of the array is larger than the first element, otherwise, less.

The key idea over here is to bound the target within the right or left interval.

Then how to bound the target into the desired interval? To keep the consistent of the interval, since the consistency implies the interval is a sequence and the algorithm can successfully drop it or move to it.

In [ ]:
class Solution:
    def search(self, nums: List[int], target: int) -> int:
        left, right = 0, len(nums) - 1

        while left <= right:
            mid = (left + right) // 2

            if nums[mid] == target:
                return mid
            elif nums[mid] < target:
                # to ensure the target is rigorly in the right interval
                if nums[left] < nums[mid] or target <= nums[right]:
                    # print('1.', left, mid, right)
                    left = mid + 1
                else:
                    # print('2.', left, mid, right)
                    right = mid - 1
            elif nums[mid] > target:
                # to ensure the target is rigorly in the left interval
                if nums[mid] < nums[right] or nums[left] <= target:
                    # print('3.', left, mid, right)
                    right = mid - 1
                else:
                    # print('4.', left, mid, right)
                    left = mid + 1

        return -1

# Footnotes

In [ ]:
# add the doc information to README 
from tools.setup import generate_row as g

g()